# Define the average Reward position on video files
Choose a path that contains one or mutliple .avi files. A window will open and the first frame of each video will be diplayed. Click on the frame where the reward position was. A red cross will appear at the location. Click again to change the location of the reward. Press "s" to save the last position and to go to the next video. The average X Y positions will be saved as a text file in the original path. Press "q" when you processed enough videos to get a good average of where the reward was.

In [ ]:
##### LOCAL or INTERACTIVE NODES #####

# Path to the root folder containing video files
video_folder = '//10.69.168.1/crnldata/forgetting/Aurelie/CheeseboardExperiment/DAQ_data/AB/'

In [ ]:
import os
import cv2
import numpy as np

# Global variables
click_position = None
frame_copy = None
quit=False

reward_postion=[]
# Mouse click event handler
def on_click(event, x, y, flags, param):
    global click_position, frame_copy
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse button click
        click_position = (x, y)
        print(f"Clicked at: ({x}, {y})")
        
        # Draw a red cross at the clicked position
        frame_copy = frame.copy()
        cv2.line(frame_copy, (x-20, y), (x+20, y), (0, 0, 255), 2)  # Horizontal line
        cv2.line(frame_copy, (x, y-20), (x, y+20), (0, 0, 255), 2)  # Vertical line

# Loop through the video folder
for root, dirs, files in os.walk(video_folder):
    video_files = [f for f in files if f.endswith(('.avi', '.mp4', '.mov'))]
    
    for video_file in video_files:
        video_path = os.path.join(root, video_file)
        video_path = os.path.normpath(video_path)

        if "My_First_WebCam" in video_path and "Cheeseboard" in video_path and 'DLC' not in video_file:
            cap = cv2.VideoCapture(video_path)

            if not cap.isOpened():
                print(f"Error: Could not open video {video_file}")
                continue

            ret, frame = cap.read()
            if not ret:
                print(f"Error: Could not read first frame of {video_file}")
                cap.release()
                continue

            frame_copy = frame.copy()
            cv2.imshow(video_path, frame_copy)
            cv2.setMouseCallback(video_path, on_click)
            click_position = None

            while quit==False:
                cv2.imshow(video_path, frame_copy)
                key = cv2.waitKey(1) & 0xFF

                if key == ord('q'):  # Quit everything
                    cap.release()
                    cv2.destroyAllWindows()
                    quit=True
                    print("Quitting program...")
                
                elif key == ord('s'):  # Skip to next video
                    reward_postion.append(click_position)
                    break

            #if click_position:
            #    save_click_position(video_path, click_position)

            cap.release()
            cv2.destroyAllWindows()
        if quit:
            break
    if quit:
        break

if reward_postion:
    txt_file_path = os.path.splitext(video_folder)[0] + "Reward_position.txt"
    with open(txt_file_path, 'w') as f:
        f.write(f"Reward Position: x={round(np.mean(reward_postion, axis=0)[0],2)}, y={round(np.mean(reward_postion, axis=0)[1],2)}")
    print(f"Saved reward position ({round(np.mean(reward_postion, axis=0)[0],2)}, {round(np.mean(reward_postion, axis=0)[1],2)}) to {txt_file_path}")

